# Applied Data Science Capstone -- Week 3
## Segmenting and Clustering Neighborhoods in Toronto -- Part 1



First, import the data table into a dataframe:

In [1]:
import pandas as pd
import numpy as np

data1 = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
data1.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


Remove records where a Borough is not assigned to a Postcode:

In [2]:
data1.drop(index=data1.loc[data1['Borough']=='Not assigned'].index, inplace=True)  # remove postcodes not assigned
data1.reset_index(inplace=True)  # reindex remaining entries
data1.drop(columns='index', inplace=True)  # remove extra index column
data1.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


If Neighborhood is not assigned, rename to Borough name:

In [3]:
renamelist=data1.loc[data1['Neighbourhood']=='Not assigned'].index
data1.iloc[renamelist,2]=data1.iloc[renamelist,1]  # for each neighborhood not assigned, rename to Borough
data1.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


If a Postcode contains multiple neighborhoods, combine them into the first postcode entry and then remove duplicate postcode entries

In [4]:
data1['Dupes']=data1.duplicated(subset='Postcode') #identify duplicate Postcodes

for i in data1.index:
    if not data1.iloc[i,3]:  # if Dupes=False, advance, keeping track if it is first duplicate entry
        k=i
    else:
        data1.iloc[k,2]=data1.iloc[k,2]+', '+data1.iloc[i,2]  #if Dupes=True, append neighborhood indo first duplicate entry
    
data1.drop(index=data1.loc[data1['Dupes']].index, inplace=True)  #remove the duplicate entries and reindex
data1.reset_index(inplace=True)
data1.drop(columns=['index','Dupes'], inplace=True)  #remove extra columns
data1.head(20)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


Print number of rows in dataframe

In [5]:
print("Number of rows in Postcode dataframe:  " + str(data1.shape[0]))

Number of rows in Postcode dataframe:  103


## Segmenting and Clustering Neighborhoods in Toronto -- Part 2

Try to get the latitude and the longitude coordinates of each Postcode using geocoder

In [7]:
import geocoder   # import the geocoder module

# initialize collectors for latitudes and longitudes
lats=[]
longs=[]
codes=[]
for i in data1.index:
    latlong_coords=None
    # loop until you get the coordinates
    k=0
    while((latlong_coords is None) and (k<20) ):
        k+=1
        # use the geocode farm instead of google, since google is denying requests
        g = geocoder.geocodefarm('{}, Toronto, Ontario'.format(data1.iloc[i,0]))
        #print(k, data1.iloc[i,0], g)
        latlong_coords = g.latlng

    #print(k, data1.iloc[i,0], g)
    codes.append(data1.iloc[i,0])
    lats.append(latlong_coords[0])
    longs.append(latlong_coords[1])
d={'Postcode':codes, 'Latitude':lats, 'Longitude':longs}
pcodes2=pd.DataFrame(data=d)
gdata2 = data1.merge(pcodes2, on="Postcode")
gdata2.head(20)


TypeError: 'NoneType' object is not subscriptable

Backup plan -- import the latitude and the longitude coordinates of the Postcodes from csv file.
This will provide more consistent results, since the geocoder input is unreliable

In [8]:
pcodes = pd.read_csv('Geospatial_Coordinates.csv')
pcodes.rename(columns={"Postal Code": "Postcode"}, inplace=True)
gdata = data1.merge(pcodes, on="Postcode")
gdata.loc[gdata['Borough']=='Central Toronto']


,Postcode,Borough,Neighbourhood,Latitude,Longitude
61,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
62,M5N,Central Toronto,Roselawn,43.711695,-79.416936
67,M4P,Central Toronto,Davisville North,43.712751,-79.390197
68,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307
73,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
74,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
79,M4S,Central Toronto,Davisville,43.704324,-79.388790
83,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
86,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


## Segmenting and Clustering Neighborhoods in Toronto -- Part 3

Explore and cluster the neighborhoods in Toronto.

In [9]:
import requests
import folium

# Pick a Toronto location for the map center
torloc = gdata.loc[gdata['Borough']=='Central Toronto'].index[0]
tlat=gdata.iloc[torloc,3]
tlong=gdata.iloc[torloc,4]

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[tlat, tlong], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(gdata['Latitude'], gdata['Longitude'], gdata['Borough'], gdata['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Too many.  Limit the locations to those boroughs nearer the downtown area

In [10]:
t=[]
for i in gdata.index:
    if 'Toronto' in gdata.iloc[i,1]:
        t.append(i)
gdatat = gdata.iloc[t]

# create map of Toronto using latitude and longitude values
torloc1 = gdata.loc[gdata['Borough']=='Central Toronto'].index[5]
tlat=gdata.iloc[torloc1,3]
tlong=gdata.iloc[torloc1,4]
map_toronto1 = folium.Map(location=[tlat, tlong], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(gdatat['Latitude'], gdatat['Longitude'], gdatat['Borough'], gdatat['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto1)  
    
map_toronto1

Enter Foursquare credentials

In [11]:
# @hidden_cell
CLIENT_ID = 'GSCGQ5LFOLW3LFHEG4GX1J0DD3IQXLBI3AK4BBFJ5TY3OGTT' # your Foursquare ID
CLIENT_SECRET = '425KMLXGMEXIV4RHUT31W2O1B03TCWGPXLRN1C3YPSUV5FRA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Foursquare credentials loaded')


Foursquare credentials loaded


Get nearby venues for all downtown neighborhoods in gdatat.

In [12]:
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(names=gdatat['Neighbourhood'], latitudes=gdatat['Latitude'], longitudes=gdatat['Longitude'])


In [14]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
toronto_venues.groupby('Neighborhood').count()

There are 234 uniques categories.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"Brockton, Exhibition Place, Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
"Cabbagetown, St. James Town",46,46,46,46,46,46
Central Bay Street,84,84,84,84,84,84
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,15,15,15,15,15,15


Prepare data for cluster analysis

In [15]:
# one hot encoding for cluster analysis
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# list neighborhoods back to first column in dataframe
toronto_onehot.drop(['Neighborhood'], axis=1, inplace=True)
toronto_onehot.insert(0,'Neighborhood',toronto_venues['Neighborhood'])
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

Print each neighborhood along with the top 5 most common venues


In [16]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.08
1             Café  0.05
2  Thai Restaurant  0.04
3       Steakhouse  0.04
4              Bar  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2                Café  0.04
3  Seafood Restaurant  0.04
4            Beer Bar  0.04


----Brockton, Exhibition Place, Parkdale Village----
                   venue  freq
0                   Café  0.09
1            Coffee Shop  0.09
2         Breakfast Spot  0.09
3  Performing Arts Venue  0.05
4          Grocery Store  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1       Garden Center  0.06
2              Garden  0.06
3       Burrito Place  0.06
4      Farmers Market  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Ser

Create a new dataframe with the top 10 venues for each neighborhood

In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,American Restaurant,Asian Restaurant,Gym,Hotel,Breakfast Spot
1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Steakhouse,Cheese Shop,Café,Farmers Market,Beer Bar,Seafood Restaurant,Shopping Mall
2,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Coffee Shop,Café,Intersection,Convenience Store,Performing Arts Venue,Pet Store,Grocery Store,Music Venue,Climbing Gym
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Gym / Fitness Center,Garden,Comic Shop,Pizza Place,Restaurant,Butcher,Burrito Place,Skate Park,Brewery
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Plane,Airport,Airport Food Court,Airport Gate,Sculpture Garden,Harbor / Marina,Boutique


Performe KMeans clustering for 5 clusters

In [18]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = gdatat

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() 

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Café,Theater,Mexican Restaurant,Restaurant,Breakfast Spot,Farmers Market
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Tea Room,Theater,Italian Restaurant,Bubble Tea Shop,Japanese Restaurant
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Hotel,Italian Restaurant,Café,Restaurant,Bakery,Cosmetics Shop,Cocktail Bar,Clothing Store,Breakfast Spot
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Trail,Pub,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Bakery,Steakhouse,Cheese Shop,Café,Farmers Market,Beer Bar,Seafood Restaurant,Shopping Mall


Map the clusters

In [19]:
# create map
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[tlat, tlong], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [20]:
#Cluster 1

toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,Coffee Shop,Park,Bakery,Pub,Café,Theater,Mexican Restaurant,Restaurant,Breakfast Spot,Farmers Market
9,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Tea Room,Theater,Italian Restaurant,Bubble Tea Shop,Japanese Restaurant
15,Downtown Toronto,0,Coffee Shop,Hotel,Italian Restaurant,Café,Restaurant,Bakery,Cosmetics Shop,Cocktail Bar,Clothing Store,Breakfast Spot
19,East Toronto,0,Health Food Store,Trail,Pub,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
20,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Bakery,Steakhouse,Cheese Shop,Café,Farmers Market,Beer Bar,Seafood Restaurant,Shopping Mall
24,Downtown Toronto,0,Coffee Shop,Ice Cream Shop,Italian Restaurant,Café,Sandwich Place,Burger Joint,Middle Eastern Restaurant,Bakery,Bar,Chinese Restaurant
25,Downtown Toronto,0,Grocery Store,Café,Park,Convenience Store,Coffee Shop,Italian Restaurant,Baby Store,Restaurant,Nightclub,Diner
30,Downtown Toronto,0,Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,American Restaurant,Asian Restaurant,Gym,Hotel,Breakfast Spot
31,West Toronto,0,Pharmacy,Supermarket,Bakery,Bank,Café,Brewery,Gym / Fitness Center,Park,Bar,Middle Eastern Restaurant
36,Downtown Toronto,0,Coffee Shop,Hotel,Aquarium,Italian Restaurant,Café,Brewery,Bakery,Scenic Lookout,Fried Chicken Joint,Pizza Place


In [21]:

#Cluster 2

toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,Central Toronto,1,Playground,Restaurant,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [22]:
#Cluster 3

toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,2,Garden,Yoga Studio,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [23]:
#Cluster 4

toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]



,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Central Toronto,3,Park,Trail,Bus Line,Jewelry Store,Sushi Restaurant,Yoga Studio,Dive Bar,Falafel Restaurant,Event Space,Ethiopian Restaurant
91,Downtown Toronto,3,Park,Playground,Trail,Building,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [24]:
#Cluster 5

toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Central Toronto,4,Construction & Landscaping,Park,Bus Line,Swim School,Dive Bar,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


Map appears to show that most of the downtown boroughs are in Cluster 1, with a higher availability of Coffee Shops and Restraunts in the top venues.  The other clusters appear to be more suburban, with higher availability of parks and playgrounds.  